In [82]:

import os
import sys
import pandas as pd
import random
import matplotlib.pyplot as plt
import sqlite3
from bs4 import BeautifulSoup, ResultSet
import urllib.request
import re
import glob
import warnings
from tqdm import tnrange, tqdm_notebook
from random import randint
import time 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

# http://efm.aeaweb.org/vivisimo/cgi-bin/query-meta?v%3aproject=econlit&v%3asources=All&&v%3aproject=econlit&v%3aframe=form&form=econlit-advanced&
# https://askubuntu.com/questions/870530/how-to-install-geckodriver-in-ubuntu
#Importing packages

warnings.filterwarnings('ignore')
random.seed(666)
os.chdir('/home/chris/HOPE/data/')

In [204]:
### CHEATING
### CHEATING

driver = webdriver.Firefox()
time.sleep(6) 
driver.get('https://libguides.liverpool.ac.uk/az.php?a=w')
time.sleep(6) 
driver.find_element_by_xpath(u'/html/body/div[2]/div/div[4]/section[2]/div/div[1]/div[3]/div/div[2]/div[2]/div[1]/a').click()
time.sleep(6) 

#driver.find_element_by_id("value(input1)").send_keys("SO = ECONOMIC JOURNAL OR SO = ECONOMETRICA OR SO = AMERICAN ECONOMIC REVIEW OR SO = JOURNAL OF POLITICAL ECONOMY OR SO = QUARTERLY JOURNAL OF ECONOMICS OR SO = REVIEW OF ECONOMIC STUDIES")
#driver.find_element_by_id("search-button").click()
#driver.find_element_by_xpath("username").send_keys("ehengel")
#driver.find_element_by_name("j_username").send_keys("adoDEM88")
#query(IS = "1468-0297") # EJ
#query(IS = "1468-0262") # ECA
#query(IS = "1944-7981") # AER
#query(IS = "1537-534X") # JPE
#query(IS = "1531-4650") # QJE
#query(IS = "1467-937X") # RES
#  SO = ECONOMIC JOURNAL OR SO = ECONOMETRICA OR SO = AMERICAN ECONOMIC REVIEW OR SO = JOURNAL OF POLITICAL ECONOMY OR SO = QUARTERLY JOURNAL OF ECONOMICS OR SO = REVIEW OF ECONOMIC STUDIES
#driver.switch_to.window(driver.window_handles[1])
#driver.find_element_by_link_text("Advanced Search").click()
#driver.find_element_by_id("value(input1)").send_keys("SO = ECONOMIC JOURNAL OR SO = ECONOMETRICA OR SO = AMERICAN ECONOMIC REVIEW OR SO = JOURNAL OF POLITICAL ECONOMY OR SO = QUARTERLY JOURNAL OF ECONOMICS OR SO = REVIEW OF ECONOMIC STUDIES")
#driver.find_element_by_id("search-button").click()
#time.sleep(6) 
#driver.switch_to.window(driver.window_handles[1])
#driver.find_element_by_xpath("markFrom").send_keys("11")
#driver.find_element_by_id("value(input1)").send_keys("IS = " + IS)



In [240]:

# Read all .txt files extracted from Web of Science
path = '/home/chris/HOPE/data/'
os.chdir('/home/chris/HOPE/data/')
files = [f for f in os.listdir(path) if os.path.isfile(f)]
data =  pd.DataFrame([])
for file in files:
    df = pd.read_table(file)
    df = df.iloc[1:,:]
    df.columns = ["metadata"]
    data = data.append(df)
    
# Make an identifying key for each idividual article
data["idx"] = ""
data.columns = ["metadata", "idx"]
counter = Article.ArticleID.max() + 1
for k in tqdm_notebook(range(0, len(data))):
    string = data.iloc[k,0]
    if string[:2] == "PT":
        data.iloc[k,-1] = int(counter)
        counter = counter +1 
data = data.replace(r'^\s*$', np.nan, regex=True)
data["idx"] = data["idx"].ffill()
data.to_csv("clean_data.csv")

# Keep all rows that ARENT already in existing .db
data['category'] = data['metadata'].str[:2]
WOS = data[data["category"] == "UT"] 
WOS["WOS"]= WOS['metadata'].str[3:]
WOS_set = set(WOS.WOS)
DOI = data[data["category"] == "DI"] 
DOI["DOI"]= DOI['metadata'].str[3:]
DOI_set = set(DOI.DOI)
new_DOI = list(DOI_set) + list(WOS_set)
master_DOI = list(set(DOICorr.DOI))
new = [x for x in new_DOI if not x in master_DOI]
DOI = DOI[["idx", "DOI"]]
WOS = WOS[["idx", "WOS"]]
WOS.columns = ["idx","DOI"]
df = DOI.append(WOS)
df = df[df['DOI'].isin(new)]
keep = list(set(df.idx))
df = data[data['idx'].isin(keep)]
df.to_csv("clean_data_2.csv")

# Reindex 
df["idx"] = ""
counter = Article.ArticleID.max() + 1
for k in tqdm_notebook(range(0, len(df))):
    string = df.iloc[k,0]
    if string[:2] == "PT":
        df.iloc[k,-2] = int(counter)
        counter = counter +1 
df = df.replace(r'^\s*$', np.nan, regex=True)
df["idx"] = df["idx"].ffill()
df.to_csv("clean_data_3.csv")
#df = pd.read_csv("clean_data_3.csv")
#df = df.drop(columns = ["Unnamed: 0"])

# Pick out individual authors
df['af_flag'] = pd.np.where(df.category.str.contains("AF"), "AF",
                            pd.np.where(df.category.str.contains("TI"), "TI",
                            pd.np.where(df.category.str.contains("AU"), "AU", "")))
df = df.replace(r'^\s*$', np.nan, regex=True)
df["af_flag"] = df["af_flag"].ffill()
df = df.replace(r'^\s*$', np.nan, regex=True)
df["idx"] = df["idx"].ffill()
authors = df[df["af_flag"] == "AF"]
authors = authors['metadata'].str[3:]
authors = pd.DataFrame(authors)
authors = authors.drop_duplicates()

# Get rid of authors that are not already in dataset
existing_authors = set(Author.AuthorName)
new_authors = set(authors.metadata)
new_authors = [x for x in new_authors if not x in existing_authors]
new_authors = pd.DataFrame(new_authors)
new_authors["AuthorID"] = range(0, len(new_authors))
new_authors["AuthorID"] = new_authors["AuthorID"] + Author.AuthorID.max()
new_authors.to_csv("new_authors.csv")

In [554]:
Author

,index,AuthorID,AuthorName,Sex,NativeLanguage,PhDYear,Note
0,0,1,A. Abigail Payne,Female,NaN,NaN,NaN
1,1,2,A. Andrew John,Male,English,NaN,NaN
2,2,3,A. Belloni,Male,NaN,NaN,NaN
3,3,4,A. Colin Cameron,Male,English,NaN,NaN
4,4,5,A. Joshua Strickland,Male,English,NaN,NaN
...,...,...,...,...,...,...,...
8205,8205,9615,Edward Tower,Male,NaN,NaN,NaN
8206,8206,9616,Andrew Feltenstein,Male,NaN,NaN,NaN
8207,8207,9617,Steven Braithwait,Male,NaN,NaN,NaN
8208,8208,9618,John S. Pettengill,Male,NaN,NaN,NaN


In [83]:
def scrape_journal(journal, start):
   
    time.sleep(10)

    driver = webdriver.Firefox()
    time.sleep(6) # lazy technical debt 
    driver.get('https://libguides.liverpool.ac.uk/az.php?a=w')
    time.sleep(6) 
    driver.find_element_by_xpath(u'/html/body/main/div/section/p[3]/a').click()
    time.sleep(6) 
    driver.find_element_by_id("logincaptchaform-username").send_keys("erin.hengel@gmail.com")
    driver.find_element_by_id("logincaptchaform-password").send_keys("wYlZYjf1e2")
    elem = driver.find_element_by_xpath('/html/body/main/div/section/form/div[2]/div/input')
    actions = ActionChains(driver)
    actions.click(elem).perform()

    time.sleep(10)


    data = pd.DataFrame([])
    string = journal # gross technical debt 
    string_length = len(string.split())
    root_start = start
     
    for i in tqdm_notebook(range(0,(2019-root_start))): # hardcoded (technical debt)
        
        
        if i > 0:
            driver.switch_to.window(driver.window_handles[0])
            start = start + 1

        
        if string_length ==  1:
            query = "so:" + str(journal) + " yearmin:"  + str(start) 
        else: 
            query = string.split()
            query = ["so:" + x  for x in query]
            query = ' '.join(query)
            query = query + " yearmin:"  + str(start) 
        print(query) 
        time.sleep(10) # Ugly workaround

        if i == 0:
            driver.find_element_by_link_text("Advanced").click()
        else:
            driver.find_element_by_link_text("Modify search").click()
    
        elem = driver.find_element_by_xpath('//*[@id="input-query"]').clear()
        elem = driver.find_element_by_xpath('//*[@id="input-source"]').clear()
        elem = driver.find_element_by_xpath('//*[@id="input-yearmin"]').clear()
        elem = driver.find_element_by_xpath('//*[@id="input-yearmax"]').clear()
        elem = driver.find_element_by_xpath('//*[@id="input-source"]').send_keys(journal)
        elem = driver.find_element_by_xpath('//*[@id="input-yearmin"]').send_keys(start)
        elem = driver.find_element_by_xpath('//*[@id="input-yearmax"]').send_keys(start)
        select = Select(driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div/form/fieldset/table/tbody/tr[4]/td/select'))
        select.select_by_value('1000')
        elem = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div/form/fieldset/div/input').click()
        time.sleep(6)
        driver.find_element_by_xpath('//*[@id="sel-all-top"]').click()
        time.sleep(6)
        driver.find_element_by_xpath('//*[@id="select-all-link"]').click()
        time.sleep(6)
        driver.find_element_by_xpath('//*[@id="export-menu-button"]').click()
        time.sleep(6)
        driver.find_element_by_xpath('/html/body/div[2]/div/div[4]/div/div/div/div[2]/div[2]/div[2]/ul/li[3]/ul/li[5]/span').click()
        time.sleep(6)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(6)
        text= driver.find_element_by_xpath("html").text
        text = pd.DataFrame([x.split(';') for x in text.split('\n')])
        mask = text.applymap(lambda x: x is None)
        cols = text.columns[(mask).any()]
        for col in text[cols]:
            text.loc[mask[col], col] = ''
        cols = text.columns
        text['combined'] = text[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
        text = pd.DataFrame(text.combined)

        # Laziest/Ugliest workaround ever. Technical debt. 
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        text['combined'] = text['combined'].str.replace(',,',',')
        for l in tqdm_notebook(range(0, len(text))):
            string = text.iloc[l,0]
            string = string.rstrip(",")
            text.iloc[l,0] = string

        data = data.append(text)
    
    return(data)

In [ ]:
Quarterly_Journal_of_Economics = scrape_journal("Quarterly Journal of Economics", 1940)
Quarterly_Journal_of_Economics.to_csv("Quarterly_Journal_of_Economics.csv")

Journal_of_Political_Economy = scrape_journal("Journal of Political Economy", 1940)
Journal_of_Political_Economy.to_csv("Journal of Political Economy.csv")

Review_of_Economic_Studies = scrape_journal("Review of Economic Studies", 1940)
Review_of_Economic_Studies.to_csv("Review of Economic Studies.csv")

Econometrica = scrape_journal("Econometrica", 1940)
Econometrica.to_csv("Econometrica.csv")

American_Economic_Review = scrape_journal("American Economic Review", 1940)
American_Economic_Review.to_csv("American Economic Review.csv")

Review_of_Economics_and_Statistics = scrape_journal("Review of Economics and Statistics", 1940)
Review_of_Economics_and_Statistics.to_csv("Review of Economics and Statistics.csv")

Journal_of_Monetary_Economics  = scrape_journal("Journal of Monetary Economics", 1975)
Journal_of_Monetary_Economics.to_csv("Journal of Monetary Economics .csv")

Journal_of_Public_Economics  = scrape_journal("Journal of Public Economics", 1973)
Journal_of_Public_Economics.to_csv("Journal of Public Economics.csv")

Journal_of_Econometrics = scrape_journal("Journal of Econometrics", 1973)
Journal_of_Econometrics.to_csv("Journal of Econometrics.csv")

Journal_of_International_Economics  = scrape_journal("Journal of International Economics", ??)
Journal_of_International_Economics.to_csv("Journal of International Economics.csv")

Journal_of_Economic_Literature  = scrape_journal("Journal of Economic Literature", 1969)
Journal_of_Economic_Literature.to_csv("Journal of Economic Literature.csv")

Journal_of_Dev_Economics  = scrape_journal("Journal of Development Economics", 1974)
Journal_of_Dev_Economics.to_csv("Journal of Development Economics.csv")

Journal_of_Financial_Economics = scrape_journal("Journal of Financial Economics", 1974)
Journal_of_Financial_Economics.to_csv("Journal of Financial Economics.csv")

International_Economic_Review = scrape_journal("International Economic Review", 1960)
International_Economic_Review.to_csv("International Economic Review.csv")

In [76]:
os.chdir("/home/chris/HOPE/") # HARDCODED 
df.to_csv("testtt.csv")

,index,AuthorID,AuthorName,Sex,NativeLanguage,PhDYear,Note
0,0,1,A. Abigail Payne,Female,NaN,NaN,NaN
1,1,2,A. Andrew John,Male,English,NaN,NaN
2,2,3,A. Belloni,Male,NaN,NaN,NaN
3,3,4,A. Colin Cameron,Male,English,NaN,NaN
4,4,5,A. Joshua Strickland,Male,English,NaN,NaN
...,...,...,...,...,...,...,...
8205,8205,9615,Edward Tower,Male,NaN,NaN,NaN
8206,8206,9616,Andrew Feltenstein,Male,NaN,NaN,NaN
8207,8207,9617,Steven Braithwait,Male,NaN,NaN,NaN
8208,8208,9618,John S. Pettengill,Male,NaN,NaN,NaN


In [58]:
os.chdir("/home/chris/HOPE/old junk/old_db/") # HARDCODED 

AuthorAlias =      pd.read_csv("AuthorAlias.csv")
JEL =              pd.read_csv("JEL.csv")
Children =         pd.read_csv("Children.csv")
Nber =             pd.read_csv("Nber.csv")
NBERCorr =         pd.read_csv("NBERCorr.csv")
NBERStat =         pd.read_csv("NBERStat.csv")
sqlite_sequence =  pd.read_csv("sqlite_sequence.csv")
Inst =             pd.read_csv("Inst.csv")
sqlite_stat4 =     pd.read_csv("sqlite_stat4.csv")
InstCorr =         pd.read_csv("InstCorr.csv")
InstAlias =        pd.read_csv("InstAlias.csv")
JournalName =      pd.read_csv("JournalName.csv")
Article =          pd.read_csv("Article.csv")
ReadStat =         pd.read_csv("ReadStat.csv")
sqlite_stat1 =     pd.read_csv("sqlite_stat1.csv")
AuthorCorr =       pd.read_csv("AuthorCorr.csv")
DOICorr =          pd.read_csv("DOICorr.csv")
Author =           pd.read_csv("Author.csv")
EditorBoard =      pd.read_csv("EditorBoard.csv")


In [84]:
os.chdir('/home/chris/HOPE/')
df = pd.read_csv("top5.csv")
df = pd.DataFrame(df.iloc[:,-1])
df["title"] = ""
df.columns = ["metadata", "title"]
df = df[~df.metadata.str.contains("This record is part")]
df = df.iloc[-1000:-1,:]
for k in tqdm_notebook(range(0, len(df))):
    string = df.iloc[k,0]
    if string[:2] == "TI":
        string = str(string)
        df.iloc[k,-1] = string
df = df.replace(r'^\s*$', np.nan, regex=True)
df["title"] = df["title"].ffill()
df = df.dropna()

In [85]:
doi = df
doi['prefix'] = df['metadata'].astype(str).str[0:2]

In [90]:
doi= doi[doi["prefix"]== "DO"]

In [23]:
df.to_csv("test.csv")

In [29]:
import numpy as np

In [92]:
df

,metadata,title,prefix
480169,TI: Just Starting Out: Learning and Equilibriu...,TI: Just Starting Out: Learning and Equilibriu...,TI
480170,"AU: Doraszelski, Ulrich, Lewis, Gregory, Pakes...",TI: Just Starting Out: Learning and Equilibriu...,AU
480171,"AF: U PA, Microsoft Research New England, Harv...",TI: Just Starting Out: Learning and Equilibriu...,AF
480172,DO: http://dx.doi.org/10.1257/aer.20160177,TI: Just Starting Out: Learning and Equilibriu...,DO
480173,"SO: American Economic Review, 108(3), March 20...",TI: Just Starting Out: Learning and Equilibriu...,SO
...,...,...,...
481216,"KY: Asymmetric Information, Information, Litig...",TI: An Empirical Analysis of the Signaling and...,KY
481217,GD: U.S.,TI: An Empirical Analysis of the Signaling and...,GD
481218,GR: Northern America,TI: An Empirical Analysis of the Signaling and...,GR
481219,LA: English,TI: An Empirical Analysis of the Signaling and...,LA


In [35]:
df


,metadata,title
10,TI: Mr. Keynes and Mr. Marx,TI: Mr. Keynes and Mr. Marx
11,"AU: Alexander, S. S.",TI: Mr. Keynes and Mr. Marx
12,AF: Unlisted,TI: Mr. Keynes and Mr. Marx
13,"SO: Review of Economic Studies, 7(0), February...",TI: Mr. Keynes and Mr. Marx
14,DT: Journal Article,TI: Mr. Keynes and Mr. Marx
...,...,...
1089,DT: Journal Article,TI: National central banking and the internati...
1090,PY: 1947,TI: National central banking and the internati...
1091,DE: International Adjustment Mechanisms: Gener...,TI: National central banking and the internati...
1092,GD: India,TI: National central banking and the internati...


In [34]:
print(len(df))
print(len(df))

999
991


In [27]:
df[['normal_price','final_price']]=df[['normal_price','final_price']].fillna(method='ffill')

,metadata,title
1,AF: Unlisted,
2,"SO: Review of Economic Studies, 8(0), February...",
3,DT: Journal Article,
4,PY: 1941,
5,DE: General Equilibrium: General 2.140,
...,...,...
1089,DT: Journal Article,
1090,PY: 1947,
1091,DE: International Adjustment Mechanisms: Gener...,
1092,GD: India,


In [ ]:
ArticleID
Journal
PubDate
Title
Abstract
Language
Volume
Issue
FirstPage
LastPage
DOI


In [286]:
# onlyine supplements 
# Online Supplement
# Report of the Editor
# Editors' Introduction
# Foreward

# Connect to SQL Database and reads every Table in the whole DB
# Then converts each table into a .csv file so we can work with themn in Python

# Top 5 Workaround (tech debt) because the original webcrawler did chunks of 2 years instead of 1, need to refactor
# Sooo.. this cell finds and deletes all those  wrongly webscraped duplicates
# Take 4 hours for loop to run... it's certainly not optimized 

#path = '/home/chris/HOPE/data/top5'
#all_files = glob.glob(path + "/*.csv")
#li = []
#for filename in all_files:
#    df = pd.read_csv(filename, index_col=None, header=0)
#    li.append(df)
#df = pd.concat(li, axis=0, ignore_index=True)
#df["duplicates"] = ""
#for k in tqdm_notebook(range(0, len(df))):
#    string = df.iloc[k,1]
###    if string[:2] == "TI":
#        string = str(string)
#        df.iloc[k,-1] = string
#df["duplicates"] = df["duplicates"].replace(r'^\s*$', np.nan, regex=True)
#df["duplicates"] = df["duplicates"].ffill()
#df.duplicates = df.combined + df.duplicates
#df = df.drop_duplicates(subset='duplicates', keep="first")
#df = df.combined
#df.to_csv("top5.csv")

# Create master dataframe

#path = '/home/chris/HOPE/'
#all_files = glob.glob(path + "/*.csv")
#li = []
#df = pd.DataFrame(df.iloc[:,-1])for filename in all_files:
#    df = pd.read_csv#(filename, index_col=None, header=0)
#    li.append(df)
#df = pd.concat(li, axis=0, ignore_index=True)
#
#def to_csv():
#
#    os.chdir("/home/chris/HOPE/") # HARDCODED 
#    db = sqlite3.connect('read.db')
#    os.chdir("/home/chris/HOPE/old_db") # HARDCODED 
#    cursor = db.cursor()
#    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#    tables = cursor.fetchall()
#    for table_name in tqdm_notebook(tables):
#        table_name = table_name[0]
#        table = pd.read_sql_query("SELECT * from %s" % table_name, db)
#        table.to_csv(table_name + '.csv', index_label='index')
#    cursor.close()
#    db.close()
#
#to_csv()

# Journal 1: AER

#def scrape_aer(base_url, latest_volume, latest_issue):
 #   
#    # Step 1: Find  urls for all issues/volumes
#    html_page = urllib.request.urlopen(base_url)
#    soup = BeautifulSoup(html_page, "html.parser")
#    links = pd.DataFrame([])
#    for link in soup.find_all('a', href=True): # Grab all the URL hyperlinks on the webpage
#        links = links.append(pd.DataFrame(pd.Series(link['href']))) 
#    links.columns = ['url']
#    links = links[links['url'].str.contains("/issues/")]
    
#    # Step 2: Find urls for all individual articles
#    papers = pd.DataFrame([])
#    for i in tqdm_notebook(range(0, len(links))): 
#        # for i in tqdm_notebook(range(0, 1)): 
#        # sleep(randint(3,10)) # Technical debt: optimize sleep timer
#3        new_url = "https://www.aeaweb.org" + str(links.iloc[i,0])
#        html_page = urllib.request.urlopen(new_url)
#        soup = BeautifulSoup(html_page, "html.parser")
#        for paper in tqdm_notebook(soup.find_all('a', href=True)): # Grab all the URL hyperlinks on the webpage
##            papers = papers.append(pd.DataFrame(pd.Series(paper['href'])))
#    papers.columns = ["url"]
#   papers = papers[papers['url'].str.contains("/articles")]
    

#    return(papers)


#df = scrape_aer(base_url = "https://www.aeaweb.org/journals/aer/issues", latest_volume = latest_aer_issue, latest_issue = latest_aer_issue)


   # return(links)
#for i in range(0, 4):
#    url = df.iloc[i,0]
#    url =  "https://www.aeaweb.org" + str(url )
#    if url[-1] == "i":
#        continue
#    print(url)
#    
#    # Turn into soup 
#    html_page = urllib.request.urlopen(url)
#    soup = BeautifulSoup(html_page, "html.parser")
#    
#    # Title
#    title  = str(soup.findAll("h1", {"class": "title"}))
#    title = title[19:-6] # hardcoded
#    
#    # Authors
#    authors = str(soup.findAll("li", {"class": "author"}))
#    authors = authors[50:-6] # hardcoded]
#    authors = pd.DataFrame(authors.split(","))
#    for j in range(0, len(authors)):
#        string = str(authors.iloc[j,0])
#        if j == 0:
#            string = string[3:-5] #harcoded
#        if j > 0: 
#            string = string.replace('</li>','')
#            string = string[50:]
#            string = string.strip()
#        authors.iloc[j,0] = string
#    
#    
#    # Authors#

    #title = title[19:-6]
    # df = title
    #print(doi)
    #title = pd.DataFrame(pd.Series(str(author[1:-1]))) # HARDCODED 
    
    # PROBLEM RE: AFFILIATION # Junk / Under-construction

#def scrape_repec(journal_name, root_url, pages, latest_volume, latest_issue):

#    """
#    This is a webcrawler that crawls through every page/article of a
#    TO DO: Make the webcrawler  "smarter" so that it can detect that there are 17 pages of data to scrape
#    TO DO: Scrape more than 5 journals
#    """#
#
#    # Soupa nd link 
#    links = pd.DataFrame([])
#    for i in tqdm_notebook(range(1, pages)):
#        url = root_url       
#        if i > 1:  # "1" workaround is due to their uniue naming convenitons
#            url = url[:-5] + str(i) + url[-5:]
#        html_page = urllib.request.urlopen(url)
#        soup = BeautifulSoup(html_page, "html.parser")
#        for link in soup.find_all('a', href=True): # Grab all the URL hyperlinks on the webpage
#            links = links.append(pd.DataFrame(pd.Series(link['href']))) 
#    links.columns = ["url"]
#    links = links[links['url'].str.contains("/a/")]
#    links = "https://ideas.repec.org" + links
#    links["metadata"] = links["url"]
#
#    # Metadata manipulation; haervests volume, issue, year, and pages of the scrape d ata
#    # TECHNICAL DEBT! These 6 lines are ugly and gross
#    links["metadata"] = links["metadata"].str.split("v", n=1, expand=True)[1]
#    links["Volume"] = links["metadata"].str.split("y", n=1, expand=True)[0]
#    links["metadata"] = links["metadata"].str.split("y", n=1, expand=True)[1]
#    links["y"] = links["metadata"].str.split("i", n=1, expand=True)[0]
#    links["metadata"] = links["metadata"].str.split("i", n=1, expand=True)[1]
#    links["Issue"] = links["metadata"].str.split("p", n=1, expand=True)[0]
##    links["metadata"] = links["metadata"].str.split("p", n=1, expand=True)[1]
#    links["FirstPage"] = links["metadata"].str.split("-", n=1, expand=True)[0]
#    links["metadata"] = links["metadata"].str.split("-", n=1, expand=True)[1]
##    links["LastPage"] = links["metadata"].str.split(".", n=1, expand=True)[0]
 #   links = links.drop(columns = ["metadata"])
 #   links["Volume"] = pd.to_numeric(links["Volume"])
 #   links = links[links["Volume"] > latest_volume] # CUTOFF FOR NEWEST ONLY 
 #   
 #   # Now we loop through all the individual URLs to scrape some more specific metadata
 #   # Question: Do citations from the original data soruce hold up?
 #   data = pd.DataFrame([]) # intialize empty dfs to be filled up in loop
 #   authors = pd.DataFrame([])
 #   for url in tqdm_notebook(links["url"]):
 #       df = links[links["url"] == url]
 #       html_page = urllib.request.urlopen(url)
 #       soup = BeautifulSoup(html_page, "html.parser")
        
        # Author(s)        
  #      author = str(soup.find(id ="author"))
  #      author = pd.DataFrame(pd.Series(str(author[1:-1]))) # HARDCODED 
        # Question - how to get institution?
        # Question - Institution 
        
  #      # Title
  #      title = str(soup.find(id ="title"))
  #      title = str(title[21:-11]) # HARDCODED 
   #     df["Title"] = title

        # Abstract        
    #    abstract = str(soup.find(id ="abstract-body"))
    #    abstract = str(abstract[24:-6]) # HARDCODED 
    #    df["Abstract"] = abstract

        # Citations
     #   citations = str(soup.find(id ="cites-tab"))
     #   if len(citations) == 4:
     #       df["CiteCount"] = 0
     #   else:
     #       citations = str(citations.split("role")[1])
     #       citations = str(citations.split("Citations")[0])
     ##       citations = str(citations.split(">")[1])
      #3#      df["CiteCount"] = citations
       # 
        # Create empty columns (quick/dirty workaround) that formats table such that you can append it to SQL 
       # df["Journal"] = journal_name
       # df["index"] = "MISSING"
       # df["ArticleID"] = "MISSING"
       # df["Language"] = "MISSING"
       # df["Received"] = "MISSING"
       # df["Accepted"] = "MISSING"
       # df["Part"] = "MISSING"
       # df["Comments"] = "MISSING"
       # df["Note"] = "MISSING"
       # df["PubDate"] = "MISSING"
       # df = df.drop(columns = ["y", "url"]) #  Technical debt 
       #3 
       #3 # Append!
       ## data = data.append(df) 
        #authors = authors.append(author)
 #   data = data[Article.columns] # HARDCODED 
 #   data = pd.DataFrame(data)
    
  #  return(data, authors)

# TOMROROW: FInish all 17 tables
#qje, authors = scrape_repec(journal_name = "QJE", root_url = "https://ideas.repec.org/s/oup/qjecon.html", pages = 10, latest_volume = latest_qje_volume, latest_issue = latest_qje_issue) 
# Write to folder (WORKAROUND!)

#os.chdir("/home/chris/HOPE/new_db") # HARDCODED 
#qje.to_csv("Article.csv")
#aer = scrape_article_urls(journal_name = "AER",root_url = "https://ideas.repec.org/s/aea/aecrev.html", pages = 10, latest_volume = latest_aer_volume, latest_issue = latest_aer_issue)
#jpe = scrape_article_urls(journal_name = "JPE",root_url = "https://ideas.repec.org/s/ucp/jpolec.html", pages = 10, latest_volume = latest_jpe_volume, latest_issue = latest_jpe_issue)
#eca = scrape_article_urls(journal_name = "ECA",root_url = "https://ideas.repec.org/s/ecm/emetrp.html", pages = 10, latest_volume = latest_eca_volume, latest_issue = latest_eca_issue)
#res = scrape_article_urls(journal_name = "RES",root_url = "https://ideas.repec.org/s/oup/restud.html.html", pages = 10, latest_jres_volume = 2, latest_issue = latest_res_issue)

#html_page = urllib.request.urlopen("https://ideas.repec.org/a/oup/qjecon/v125y2010i2p729-765..html")
#soup = BeautifulSoup(html_page, "html.parser")
#        # Author(s)        
#author = str(soup.find(id ="listed-authors"))
#print(author)
#author = pd.DataFrame(pd.Series(str(author[1:-1]))) # HARDCODED 
#authorr = pd.DataFrame(pd.Series(str(author.split("authorname",1))))
#authorr.iloc[0,0] = authorr.iloc[0,0][82:-20] #HARDCODED (gross)
#
#print(authorr.iloc[0,0])
#
#    print(authorr[i])
# JEL Codes per Journal - how have they evolved over time?
# Technichal debt - can be made much sleeker
#set(JournalName.Journal)
#ECA_articles = set(Article[Article["Journal"] == "ECA"].iloc[:,1])
#AER_articles = set(Article[Article["Journal"] == "AER"].iloc[:,1])
#JPE_articles = set(Article[Article["Journal"] == "JPE"].iloc[:,1])
#RES_articles = set(Article[Article["Journal"] == "RES"].iloc[:,1])
#QJE_articles = set(Article[Article["Journal"] == "QJE"].iloc[:,1])
#top5_articles = [ECA_articles, AER_articles, JPE_articles, RES_articles, QJE_articles]

# Make flags for every JEL code via loop
# I figure we might need these flags for fixed effects or something at some point
#df = Article # I just like working with df better
#articles = set(Article.ArticleID)
#JEL_codes =
#for article in tqdm(articles):

# Now split this graph into two
#    chunk = JEL[JEL["ArticleID"] == article]
#    moving average of JEL codes per year per journal
#
#    print(df)
#    print(article)

# That's a picture for the entier thign, yes but how have they evolved over time?
# 1 Pie Chart Per Decade Per Journal of JEL Codes
# Investigate articles with zero JEL codes. Is that something I have to go get?

#JEL
#JEL_tally = JEL.groupby("JEL").count()
#JEL_tally = JEL_tally.iloc[:,1]
#print( JEL_tally )

# The growth of the averga page length over time

# Difference in page length between men and women

# Exploratory regressions

# Descriptive Statistics

# Break it down by Author Order

# Productiviy per Age -  broken down by Gender
    # Metric 1: Average number of citations/publications for each age

# Quantifying Dynamics of Gendered Network Formation
# First turn everything into nodes and edges

#nodes = JEL
#https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python


#c = conn.cursor()

# New Journal IDs
# Create table
#c.execute('''CREATE TABLE stocks
#             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
# c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
#conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
#conn.close()
    #print("Landing Page...")
    #driver.get('https://www.aeaweb.org/econlit/')
    #driver.find_element_by_link_text("Log In").click()
    #time.sleep(5) 
    #print("Dashboard..")
    #driver.find_element_by_id("logincaptchaform-username").send_keys("erin.hengel@gmail.com")
    #3driver.find_element_by_id("logincaptchaform-password").send_keys("wYlZYjf1e2")
    #elem = driver.find_element_by_xpath('/html/body/main/div/section/form/div[2]/div/input')
    #actions = ActionChains(driver)
    #actions.click(elem).perform()
    #time.sleep(5) 
    #print("AEA for Members..")
    #driver.get('http://efm.aeaweb.org/vivisimo/cgi-bin/query-meta?v:project=econlit&v:frame=form&frontpage=1')
    #driver.quit()
    
    
    
#def find_latest(journal):
 #   
#    """
#    This function finds the "newest" metadata and saves it as variables that we use later on
#    We then use these variables later on to automate thresholds and cutoffs
#    This is done bcause we specficially do NOT WANT to grab any data prior to our cutoff
#    """
#    
#    data = Article[Article["Journal"] == journal]
#    data = data.sort_values(by=["Volume", "Issue"])
 #   data = pd.DataFrame(data.iloc[-1,:])
#    data = data.T
#    volume = int(data["Volume"])
#    issue = int(data["Issue"])

 #   return(volume, issue)

#latest_eca_volume, latest_eca_issue = find_latest("ECA")
#latest_aer_volume, latest_aer_issue = find_latest("AER")
#latest_jpe_volume, latest_jpe_issue = find_latest("JPE")
#latest_qje_volume, latest_qje_issue = find_latest("QJE")
#latest_res_volume, latest_res_issue = find_latest("RES")